In [1]:
import pandas as pd
import numpy as np

import Bio.PDB
from Bio import PDB
from Bio.PDB.PDBIO import PDBIO as io
import os

Ha valamiért nem importálná az egész PDB sublibrary-t
import Bio.SVDSuperimposer
from Bio.PDB import Superimposer
from Bio.PDB.PDBIO import PDBIO

In [2]:
basedir = "/home/takacsg/Documents/sh2db/data/"

# Import "Pfam" table:

### Without negative values

In [21]:
# pfam = pd.read_csv(basedir+'data/SH2_domain_containing_prot_right_resnum_with_pdb_nums_domcol_0503.csv')
pfam = pd.read_csv(basedir+'pfam_table_1017.csv')
pfam.drop(['Unnamed: 0'], 'columns', inplace = True)
#pfam.drop(['Unnamed: 0.1'], 'columns', inplace = True)
pfam

,UniProt entry,Gene_col,Gene_domain,Pole,UniProt residues,PDB ID,PDB chain ID,PDB residues,Category,Gene name,...,Res_stop,Uniprot_ID,New_uniprot_start,New_uniprot_stop,Old_uniprot_start,Old_uniprot_stop,diff_start,New_PDB_start,diff_stop,New_PDB_stop
0,3BP2_HUMAN,3BP2,SH3BP2,N,458 - 538,2CR4,A,20 - 100,SigReg,SH3BP2,...,100,P78314,435,561,458,538,23,0,23,123
1,ABL1_HUMAN,ABL1,ABL1,N,127 - 202,1AB2,A,11 - 86,Kinases,ABL1,...,86,P00519,105,228,127,202,22,0,26,112
2,ABL1_HUMAN,ABL1,ABL1,N,127 - 202,1OPL,A,146 - 221,Kinases,ABL1,...,221,P00519,105,228,127,202,22,124,26,247
3,ABL1_HUMAN,ABL1,ABL1,N,127 - 202,1OPL,B,146 - 221,Kinases,ABL1,...,221,P00519,105,228,127,202,22,124,26,247
4,ABL1_HUMAN,ABL1,ABL1,N,127 - 202,2ABL,A,146 - 221,Kinases,ABL1,...,221,P00519,105,228,127,202,22,124,26,247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,ZAP70_HUMAN,ZAP70,ZAP70_C,C,163 - 239,1M61,A,163 - 239,Kinases,ZAP70,...,239,P43403,148,265,163,239,15,148,26,265
636,ZAP70_HUMAN,ZAP70,ZAP70_C,C,164 - 239,2OQ1,A,165 - 241,Kinases,ZAP70,...,241,P43403,148,265,164,239,16,149,26,267
637,ZAP70_HUMAN,ZAP70,ZAP70_C,C,165 - 239,2OZO,A,163 - 239,Kinases,ZAP70,...,239,P43403,148,265,165,239,17,146,26,265
638,ZAP70_HUMAN,ZAP70,ZAP70_C,C,167 - 239,4XZ0,A,163 - 239,Kinases,ZAP70,...,239,P43403,148,265,167,239,19,144,26,265


# Import table from alignment:

In [4]:
table = pd.read_csv(basedir+'table_from_alignment_with_pdbs_0917.csv', index_col = [0,1])
table = table.fillna('')
table

2  3  4  5  6  7    8    9   10   11  ... 254 255 256  \
0       1                                                   ...               
NaN     NaN                                                 ...               
        NaN                                                 ...               
ABL1    nums                            105  106  107  108  ...   -   -   -   
        seq           -  -  -  -  -  -    K    N    G    Q  ...   -   -   -   
        1AB2_A_SH2_N                                        ...               
...                  .. .. .. .. .. ..  ...  ...  ...  ...  ...  ..  ..  ..   
ZAP70_N 2OQ1_A_SH2_N                                        ...               
        2OZO_A_SH2_N                                        ...               
        4K2R_A_SH2_N                                        ...               
        4XZ0_A_SH2_N                                        ...               
        4XZ1_A_SH2_N                                        ...               

                     257 258 259 260 261 262 263  
0       1                                         
NaN     NaN                                       
        NaN                                       
ABL1    nums           -   -   -   -   -   -      
        seq            -   -   -   -   -   -      
        1AB2_A_SH2_N                              
...                   ..  ..  ..  ..  ..  ..  ..  
ZAP70_N 2OQ1_A_SH2_N                              
        2OZO_A_SH2_N                              
        4K2R_A_SH2_N                              
        4XZ0_A_SH2_N                              
        4XZ1_A_SH2_N                              

[877 rows x 262 columns]

# Structural alignment:

In [5]:
core_unit_tmp = list(range(48,214))
secondary_motifs_tmp = [48,49,50,51,
                   59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,
                   85,86,87,88,89,90,91,92,
                   100,101,102,103,104,105,106,107,108,109,110,
                   129,130,131,132,133,134,135,136,137,138,139,140,
                   162,163,164,165,166,167,168,169,170,171,172,
                   176,177,178,179,
                   182,183,184,185,186,187,188,189,190,191,192,
                   208,209,210,211,212,213]

gaspar_0902 = [str(i) for i in list(range(48,52))+list(range(59,77))+list(range(84,92))+list(range(100,110))]

core_unit = [str(i) for i in core_unit_tmp]
secondary_motifs = [str(i) for i in secondary_motifs_tmp]

In [6]:
gaspar_0917 = list(table.columns[table.iloc[0] != ''])
print(gaspar_0917)

['48', '49', '50', '51', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '83', '84', '85', '86', '87', '88', '89', '90', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '164', '165', '166', '167', '168', '180', '181', '182', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208']


In [28]:
errors = []
RMSDS = {}
io = PDB.PDBIO()
parser = PDB.PDBParser(QUIET=True)
super_imposer = Bio.PDB.Superimposer()

target_structure = parser.get_structure('2KK6_A_SH2_N', '/home/takacsg/Documents/sh2db/Structures/Kinases/FER/2KK6/2KK6_A_SH2_N_R.pdb')
target_model = target_structure[0]
target_residues = PDB.Selection.unfold_entities(target_model, "R")
#print('target_residues', target_residues)

counter = 0
for cat in pfam["Category"].unique():
    for gene in pfam[pfam["Category"] == cat]["Gene_domain"].unique():
        for pdb in pfam[pfam["Gene_col"] == gene]["PDB ID"].unique():
            for chain in pfam[pfam["PDB ID"] == pdb]["PDB chain ID"].unique():
                for pole in pfam[(pfam["PDB ID"] == pdb) & (pfam["PDB chain ID"] == chain) & (pfam["Gene_domain"] == gene)]["Pole"].unique():
                    counter += 1
                    # 0. Create a variable for identifying the pdb files
                    pdb_id = str(pdb+'_'+chain+'_SH2_'+pole)
                    print(pdb_id)
                    
                    
                    # 1. 2 pdb to compare them
                    target = table.loc[('FER' , '2KK6_A_SH2_N')]
                    try: 
                        query = table.loc[(gene ,pdb_id)]
                    except KeyError:
                        continue
                    #print("Q", query)
                    #print("T", target)
                    
                    
                    # 2. Take the cells where there is an aminoacid: 
                    # Target is the sequence what will be used as a template in the alignment - STAT5B
                    target_indices=[idxi for (idxi,i) in zip(query.keys(),query.values[0]) if i in ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']]
                    # Query is the structure what will be aligned to the target structure
                    query_indices=[idxi for (idxi,i) in zip(target.keys(),target.values[0]) if i in ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']]
                    
                    
                    # 3. take the intersection of the two lists
                    common_indices_tmp = [i for i in target_indices if i in query_indices]
                    
                    print("common_indices_tmp", common_indices_tmp)
                    
                    #### NEW #### Optionally restrict *common_indices* to core unit or secondary motifs
                    ## UNCOMMENT ONE OF THESE LINES
                                        
                    #common_indices = common_indices_tmp
                    #common_indices = [i for i in common_indices_tmp if i in core_unit]
                    #common_indices = [i for i in common_indices_tmp if i in secondary_motifs]
                    common_indices = [i for i in common_indices_tmp if i in gaspar_0917]
                    
                    print("common_indices", common_indices)

                    
                    # 4. Searching for the residue numbers and create two lists for the structure alignment input
                    target_seq = table.loc[('FER','nums')][common_indices].values[0]
                    query_seq = table.loc[(gene,'nums')][common_indices].values[0]
                    #print("len target", len(target_seq))
                    #print("len query", len(query_seq))
                    
                    # EZEK ÜRESEK:
                    print("target_seq", target_seq)
                    print("query_seq", query_seq)
                    
                    
                    # 5. Load the PDB structures:
                    if os.path.exists('/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'_R.pdb'):
                        file_name =  '/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'_R.pdb'
                    else: 
                        file_name = '/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene[:-2]+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'_R.pdb'
                    
                    query_structure = parser.get_structure(pdb_id, file_name)
                    query_model = query_structure[0]
                                        
                    query_residues = PDB.Selection.unfold_entities(query_model, "R")
                    
                    #print('query_residues', query_residues)
                                        
                    # 6. get the list of the atoms from the loaded stucture:
                    
                    target_atoms = []                    
                    # Iterate all in the STAT5B model to find all residues
                    for target_chain in target_model:
                        for target_res in target_chain:
                            #print("target RES", "\n", target_res)
                            #print("target get id", target_res.get_id()) # ITT MÉG MEG VANNAK A RESIDUE-K
                            # If the STAT5B residue is in the query_seq add it to the STAT5B atoms list
                            if str(target_res.get_id()[1]) in target_seq:
                                #print("target get id", target_res.get_id())
                                target_atoms.append(target_res['CA'])
                    
                    query_atoms = []                    
                    for query_chain in query_model:
                        for query_res in query_chain:
                            #print("QUERY RES", "\n", query_res)
                            #print("query get id", query_res.get_id()) # ITT MÉG MEG VANNAK A RESIDUE-K
                            if str(query_res.get_id()[1]) in query_seq:
                                #print("query get id", query_res.get_id())
                                query_atoms.append(query_res['CA'])

                    # EZEK MÁR ÜRESEK:            
                    #print('target', "\n", target_atoms)
                    #print('query', "\n", query_atoms)
                    #print('target_atoms',len(target_atoms))
                    #print('query_atoms', len(query_atoms))
                    
                    # Superimpose the structures:
                    
                    # Specify the atom lists
                    # 'query_seq' and 'target_seq' are lists of Atom objects
                    # The target_seq atoms will be put on the query_seq
                    try:
                        super_imposer.set_atoms(target_atoms, query_atoms)
                    except:
                        errors.append(query)
                        continue
                    ###super_imposer.set_atoms(target, query)
                    
                    # Apply rotation/translation to the target_seq atoms
                    super_imposer.apply(query_structure.get_atoms())
                    # Print rotation/translation & RMSD
                    print("rotran", super_imposer.rotran)
                    rmsd = super_imposer.rms
                    print("rmsd", super_imposer.rms)
                    
                    # SAVE THE RMSD VALUES INTO A DICTIONARY:
                    RMSDS[pdb_id] = rmsd
                    
                    # Save the aligned structure
                    #io.save(basedir+'Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'_R.pdb') # MAPPA!! +pdb+'_1_'+chain+'_SH2_'+pole+'_2.pdb'
                    io.set_structure(query_structure)
                    io.save('/home/takacsg/Documents/sh2db/Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'_SUPERPOSED.pdb') # MAPPA!!
                    

RMSDS_df = pd.DataFrame.from_dict(RMSDS, orient='index')
print(RMSDS_df)
print(counter)

1NRV_A_SH2_N
common_indices_tmp ['26', '27', '28', '38', '41', '42', '48', '49', '50', '51', '52', '53', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '96', '97', '98', '99', '100', '101', '102', '103', '104', '108', '126', '130', '131', '132', '133', '134', '135', '137', '138', '140', '141', '166', '167', '168', '169', '178', '179', '180', '181', '182', '183', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '211', '212', '221', '222', '224', '225', '227', '230', '231']
common_indices ['48', '49', '50', '51', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '83', '84', '85', '86', '87', '88', '89', '90', '98', '99', '100', '101', '102', '103', '104', '130', '131', '132', '133', '134', '135', '137', '138', '140', '141', '166', '167', '168', '180', '181', '182', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204']


/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:59: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: PerformanceWarning: indexing past lexsort d

rotran (array([[-0.93915277,  0.10836736, -0.32595795],
       [ 0.25135242,  0.86357995, -0.43709454],
       [ 0.23412397, -0.49242887, -0.8382719 ]]), array([-1.01465538,  0.23452163, -1.27078371]))
rmsd 3.4998197460835203
2QMS_A_SH2_N
common_indices_tmp ['20', '21', '22', '24', '26', '27', '28', '38', '41', '42', '48', '49', '50', '51', '52', '53', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '108', '126', '130', '131', '132', '133', '134', '135', '137', '138', '140', '141', '144', '163', '166', '167', '168', '169', '178', '179', '180', '181', '182', '183', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '211', '212', '221', '222', '224', '225', '227', '230', '231', '233', '234']
common_indices ['48', '49', '50', '51', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '83', 

/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:59: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: PerformanceWarning: indexing past lexsort d

rotran (array([[ 0.08534222,  0.38036762, -0.92088934],
       [-0.49510625, -0.78588129, -0.37048671],
       [-0.86463085,  0.48755623,  0.12125353]]), array([-10.97993435,  -4.52591013,  21.7438213 ]))
rmsd 2.5149543720375154
3PQZ_D_SH2_N
common_indices_tmp ['41', '42', '48', '49', '50', '51', '52', '53', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '108', '126', '130', '131', '132', '133', '134', '135', '137', '138', '140', '141', '144', '166', '167', '168', '169', '178', '179', '180', '181', '182', '183', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '211', '212', '221', '222', '224', '225', '227']
common_indices ['48', '49', '50', '51', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '83', '84', '85', '86', '87', '88', '89', '90', '98', '99', '100', '101', '102', '103'

...

/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:59: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: PerformanceWarning: indexing past lexsort d

rotran (array([[ 0.17545003, -0.62441551, -0.76113242],
       [ 0.98444346,  0.10389309,  0.14169437],
       [-0.00939977, -0.77415211,  0.63292982]]), array([ -5.03463566,   7.49225251, -18.495752  ]))
rmsd 2.070157532151149
4Y5U_B_SH2_C
common_indices_tmp ['21', '22', '24', '25', '26', '27', '28', '39', '40', '41', '42', '48', '49', '50', '51', '52', '53', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '108', '126', '130', '131', '132', '133', '134', '135', '137', '193', '194', '195', '196', '197', '198', '199', '200', '201', '224', '225', '227', '231', '232', '235', '236', '237', '238', '239', '240', '241', '242', '244']
common_indices ['48', '49', '50', '51', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '83', '84', '85', '86', '87', '88', '89', '90', '98', '99', '100', '101', '102', '103', '104', '105', '

/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:59: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: PerformanceWarning: indexing past lexsort d

rotran (array([[ 0.76542077, -0.04632457, -0.64186064],
       [ 0.36905083, -0.78549715,  0.49678538],
       [-0.52719307, -0.61712905, -0.58413971]]), array([33.30018729, 25.18183611, 20.40888252]))
rmsd 2.04626523953256
5D39_D_SH2_C
common_indices_tmp ['21', '22', '24', '25', '26', '27', '28', '39', '40', '41', '42', '48', '49', '50', '51', '52', '53', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '108', '126', '130', '131', '132', '133', '134', '135', '137', '193', '194', '195', '196', '197', '198', '199', '200', '201', '224', '225', '227', '231', '232', '235', '236', '237', '238', '239', '240', '241', '242', '244']
common_indices ['48', '49', '50', '51', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '83', '84', '85', '86', '87', '88', '89', '90', '98', '99', '100', '101', '102', '103', '104', '105', '130'

/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: PerformanceWarning: indexing past lexsort depth may impact performance.
/home/takacsg/.local/lib/python3.6/site-packages/ipykernel_launcher.py:59: PerformanceWarning: indexing past lexsort depth may impact performance.
